In [1]:
%gui qt
from mayavi import mlab
import numpy as np
import time
import os


In [ ]:
# for testing
mlab.test_contour3d_anim()

In [2]:
DATA_SET = "data_multi_1"

Now, the numpy array is num_frames x 3 x number of satellites

In [3]:
# Tmp code to mix all three bodies
data0 = data = np.load(f"data/data_0.npy")
data1 = data = np.load(f"data/data_1.npy")
data2 = data = np.load(f"data/data_2.npy")

print(data0.shape)
print(data1.shape)
print(data2.shape)

(525600, 3)
(525600, 3)
(525600, 3)


In [4]:

# DATA_SCALE_FACTOR = radius_earth
DATA_SCALE_FACTOR = 6.378e6

# Lets divide everything by R, to get intution
new_shape = list(data0.shape)
new_shape.append(3)

data = np.empty(shape = new_shape)

data[:, : , 0] = data0
data[:, : , 1] = data1
data[:, : , 2] = data2
data = data/DATA_SCALE_FACTOR


print(data.shape)
print(data[:10])

(525600, 3, 3)
[[[1.9986187  1.9986187  1.9986187 ]
  [0.05256051 0.06437321 0.07433178]
  [0.         0.         0.        ]]

 [[1.99585561 1.99585633 1.99585704]
  [0.10508468 0.12870195 0.14861223]
  [0.         0.         0.        ]]

 [[1.99171123 1.99171552 1.9917198 ]
  [0.15753612 0.19294179 0.22279025]
  [0.         0.         0.        ]]

 [[1.98618699 1.98620128 1.98621549]
  [0.20987842 0.25704858 0.29681535]
  [0.         0.         0.        ]]

 [[1.97928531 1.97932093 1.97935627]
  [0.26207517 0.32097858 0.37063798]
  [0.         0.         0.        ]]

 [[1.97100955 1.97108413 1.97115783]
  [0.31408998 0.38468861 0.44420984]
  [0.         0.         0.        ]]

 [[1.96136406 1.96150275 1.96163921]
  [0.36588652 0.44813622 0.51748419]
  [0.         0.         0.        ]]

 [[1.95035415 1.95059083 1.95082256]
  [0.41742852 0.51127985 0.59041611]
  [0.         0.         0.        ]]

 [[1.9379861  1.93836451 1.93873293]
  [0.46867985 0.57407895 0.66296277]
  [0.  

In [5]:
# Frame of reference setup
frame_x, frame_y, frame_z = np.mgrid[-10: 10: 2j,-10: 10: 2j,-10: 10: 2j]

mlab.clf()

# Uncomment the below line to make endpoints invisible
# mlab.points3d(frame_x, frame_y, frame_z, opacity=0)
mlab.points3d(frame_x, frame_y, frame_z, opacity=0.0)


In [6]:

# So as to get spatial visualization in mayavi
VISUAL_SCALE_FACTOR = 3

In [10]:
earth_obj = mlab.points3d(0,0,0, color = (1,0,0))


sat_init_pos = data[0] * VISUAL_SCALE_FACTOR
# size is 3 * num_satellites
print((sat_init_pos[0]))
print(sat_init_pos[1])
print(sat_init_pos[2])

sat_objs = mlab.points3d(
        sat_init_pos[0],
        sat_init_pos[1],
        sat_init_pos[2],
        # color = (0.5, 0.5, 0.5)
        scale_factor = 1
    )
    

# print(sat_init_pos)


sat_trajs =[
    mlab.plot3d(
        sat_init_pos[0, i],
        sat_init_pos[1, i],
        sat_init_pos[2, i],
    )

    for i in range(data.shape[2])

]
print(sat_trajs)


[5.99585609 5.99585609 5.99585609]
[0.15768153 0.19311964 0.22299535]
[0. 0. 0.]
[<mayavi.modules.surface.Surface object at 0x0000022180807950>, <mayavi.modules.surface.Surface object at 0x0000022199A704F0>, <mayavi.modules.surface.Surface object at 0x0000022199A6A860>]


In [8]:
# Lets once try mlab animate decorator...
# If it doesn't work, do with save_fig and delay and all
# @mlab.animate

# Not using decorator here, as I need inputs to this function...

def anim(moving_obj, data, num_frames = 50, obj_trajs = None, pics_folder_name = None):

    # decorator not working... Animation stops in between
    # Now again,it doesn't stop

    delay = 100

    # @mlab.animate(delay = 100)

    if not os.path.exists(pics_folder_name):
                os.makedirs(pics_folder_name)
    if pics_folder_name is None:
         pics_folder_name = "pics"

    for i in range(num_frames):
        moving_obj.mlab_source.x = data[i][0] * VISUAL_SCALE_FACTOR  
        moving_obj.mlab_source.y = data[i][1] * VISUAL_SCALE_FACTOR 
        moving_obj.mlab_source.z = data[i][2] * VISUAL_SCALE_FACTOR 

        # Trajectory is not working, bodies just dont animate ...

        if obj_trajs is not None:
            for traj_idx, obj_traj in enumerate(obj_trajs):
                obj_traj.mlab_source.reset(
                    x = data[:i+1, 0, traj_idx] * VISUAL_SCALE_FACTOR,
                    y = data[:i+1, 1, traj_idx] * VISUAL_SCALE_FACTOR,
                    z = data[:i+1, 2, traj_idx] * VISUAL_SCALE_FACTOR
                )

        # Uncomment below line, if using decorator
        # yield
        # uncomment below lines, if not using decorator
        mlab.process_ui_events()
        # This savefig is a cruel trick to re-create the image or smthing like that
        mlab.savefig('./{}/anim_{:03}.png'.format(pics_folder_name, i))

        # time.sleep(delay/1000)



In [9]:
num_frames_interest = 200


anim(sat_objs, data, 
     num_frames = num_frames_interest, 
     obj_trajs = sat_trajs,
     pics_folder_name =  f"pics/{DATA_SET}/"
       )


# Obtained good animation results! Creates a folder with pic names for the same